# Random sampling and Monte Carlo methods

Authors: Maya Fishbach, adapted from Carl Haster

In [ ]:
import numpy as np

import matplotlib 
%matplotlib inline
%config InlineBackend.figure_format='retina'
matplotlib.rcParams['figure.figsize'] = (5, 3)
import matplotlib.pyplot as plt

from scipy.integrate import cumulative_trapezoid as cumtrapz

## Why samples?

* maximum likelihood is not enough for many applications, especially in high dimensions
  * the probability contained in a given region of parameter space is the probability density $\times$ volume
  * the volume can be very large away from the maximum likelihood or maximum a posteriori
* samples allow us to compute expectation values ("Monte Carlo integrals")
  * moments of the distribution (mean, variance, etc.)
* map out the probability distribution, find correlations, etc.
  * sampling from the distribution can be much cheaper than evaluating the likelihood on a high-dimensional grid!

## Drawing samples from a PDF

* pseudo-random number generators
* Given a (pseudo)-random number from a known distribution, transform to a different distribution via:
  * inverse transform sampling
  * rejection sampling

### Psuedo-random number generators

Computers are deterministic. A pseudo-random number generator uses some formula whose outcome depends sensitively on the input, then iterate.

Note: Need starting point for the iteration (random number seed), but that makes the result reproducible (good for testing and debugging).

A popular pseudo-random number generator is a [linear congruential generator](https://en.m.wikipedia.org/wiki/Linear_congruential_generator). Such a generator is implemented below.

In [ ]:
def my_rng(m=2**32, a=1103515245, b=12345):
    """Generate a sequence of (pseudo-)random numbers from a uniform distribution.
    This function updates a given (pseudo-)random number to a new one.
    """
    my_rng.current = (a*my_rng.current + b) % m
    return my_rng.current/m

In [ ]:
my_rng.current = 239485821 #setting the random number seed
random_numbers = np.array([my_rng() for i in range(1000)]) #Draw a 1000 random numbers
plt.hist(random_numbers,range=(0.,1.),bins=10) #plot a histogram
plt.xlabel("random number")
plt.ylabel("counts")
plt.show()

In numpy, this is implemented in the [numpy.random](https://numpy.org/doc/stable/reference/random/index.html) module. 

In [ ]:
rng = np.random.default_rng(seed = 123)
# Generate 1000 random floats uniformly distributed over the range [0, 1)
random_uniform = rng.random(1000) 
plt.hist(random_uniform,range=(0.,1.),bins=10) #plot a histogram
plt.xlabel("random number")
plt.ylabel("counts")
plt.show()

### Inverse transform sampling

If we can draw random numbers from a uniform distribution, we can draw from any distribution as long as we can calculate its CDF. 

* Calculate cumulative probability distribution (CDF): $\mathrm{CDF}(x) = \int_{-\infty}^x \mathrm{d}x' P(x)$
* invert that function to get the inverse cumulative probability distribution (iCDF) such that if y = CDF(x), iCDF(y) = x
* draw random values $y$ between 0 and 1
* evaluate $\mathrm{iCDF}(y)$ to find values of $x$ following the distribution $P(x)$

For example, we can take our uniformly distributed random numbers and transform them to a set of random draws from a Gaussian distribution.

In [ ]:
def Gaussian_pdf(x, mu = 0, sigma = 1):

    return np.exp(-(x-mu)**2/(2*sigma**2)) * (2*np.pi*sigma**2)**-0.5

#the Gaussian cdf is known analytically, but for other distributions we may have to numerically integrate

xs = np.linspace(-5,5,1000) #define array; +/- 5 sigma should capture almost 100% of the probability
pdf_xs = Gaussian_pdf(xs) #evaluate Gaussian pdf on array 
cdf_xs = cumtrapz(pdf_xs, xs, initial = 0) #take the cumulative integral of the pdf to calculate the cdf
inv_cdf = lambda y: np.interp(y, cdf_xs, xs) #calculate the inverse function of the cdf; for any number y between 0 and 1, return iCDF(y)

#transform our 1000 draws from a uniform distribution (saved in random_uniform array)
random_normal_its = inv_cdf(random_uniform)

#plot 
plt.hist(random_normal_its,bins=20, density = True) #plot a histogram
plt.plot(xs, pdf_xs)
plt.xlim(-4,4)
plt.ylim(0,0.5)

plt.xlabel('random number')
plt.ylabel('pdf')

plt.show()

Numpy already has a built in function to draw from a Gaussian distribution:

In [ ]:
# Generate an array of 1000 random floats drawn from a standard normal (Gaussian) distribution
rng = np.random.default_rng(seed = 123)
random_normal_np = rng.standard_normal(10000) 

plt.hist(random_normal_np,bins=20, density = True) #plot a histogram

plt.plot(xs, pdf_xs) #plot analytic pdf

plt.xlim(-4, 4)
plt.ylim(0, 0.5)

plt.xlabel("random number")
plt.ylabel("pdf")
plt.show()

## Rejection sampling

In more than one dimension, there is no well-defined cumulative distribution function. Thus, we have to come up with a new idea:

* Sample (some region of) N-dimensional space following a simple PDF $Q(x)$ (we will use a uniform distribution).
* Add an acceptance-rejection step that accepts the sample $x$ with probability $P(x)/(c\ Q(x))$.
* Choose $c$ such that $P(x) < c\ Q(x)$ for all $x$ (but ideally not too large).

We will use a two-dimensional Gaussian in this example.

In [ ]:
#Set up a two-dimensional grid for plotting purposes:
xvals = np.arange(-10,10.01,0.01)
yvals = np.arange(-10,10.01,0.01)
gridx, gridy = np.meshgrid(xvals,yvals)

#Define the two-dimensional covariance matrix:
cov_xx = 6.
cov_yy = 1.5
cov_xy = 1.0
cov = np.array([[cov_xx,cov_xy],[cov_xy,cov_yy]])
invcov = np.linalg.inv(cov) #precompute the inverse covariance matrix (because it can get expensive)

#define a two-dimensional Gaussian centered at 0 with a given covariance matrix
def twoDGauss(samp,cov,invcov):
    x = samp[0] 
    y = samp[1]
    det = np.linalg.det(cov)
    norm = 1./(2.*np.pi)/det**0.5
    return norm*np.exp(-0.5*(x*(invcov[0,0]*x + invcov[0,1]*y) + y*(invcov[1,0]*x + invcov[1,1]*y)))

#Evaluate the Gaussian at all grid points and plot it:
G = twoDGauss([gridx,gridy],cov,invcov)

plt.imshow(G,cmap=plt.cm.Blues,extent=[xvals.min(),xvals.max(),yvals.max(),yvals.min()])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(label = r'$p(x,y)$')
plt.show()

In [ ]:
#Define drawing step from proposal distribution:
def draw_sample(rng = np.random.default_rng(seed = 123)):
    return rng.uniform(low = -10, high = 10, size = 2) #two-dimensional uniform distribution

#Define the acceptance-rejection step:
c = 40 #as defined in equations above
D = 2 #dimension of the parameter space

def AcceptReject(c,D,P,*Pargs, rng = np.random.default_rng(seed = 123)):
    """This function returns `True' if the sample is accepted and `False' if not. We use a
    variable-length argument list `*Pargs' to be able to use any probability function `P' that we
    might come up with"""
    Pval = P(*Pargs)
    proposalval = 1/20**D #The proposal density is 1/20*1/20
    prob = Pval/(c*proposalval) #acceptance probability
    return rng.choice([True,False],p=[prob,1-prob]) #accept with probability prob, reject with probability 1-prob

In [ ]:
#Draw some samples from the proposal distribution:
N = 10000 #number of samples to draw
count = 0 #count the accepted samples
samples = [] #store the accepted samples

for i in range(N):
    samp = draw_sample() #draw a sample from the proposal distribution
    acc = AcceptReject(c,D,twoDGauss,samp,cov,invcov)
    if acc:
        count += 1
        samples.append(samp)
samples = np.array(samples)

plt.imshow(G,cmap=plt.cm.Blues,extent=[xvals.min(),xvals.max(),yvals.max(),yvals.min()])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(label = r'$p(x,y)$')
plt.plot(samples[:,0],samples[:,1],'.',color='red', alpha = 0.3)
plt.show()
print('acceptance ratio:', count/N)

Let's calculate some moments of the PDF:

In [ ]:
samples = np.array(samples)
print('mean x:', np.mean(samples[:,0]))
print('mean y:', np.mean(samples[:,1]))
print('variance in x-direction:', np.mean(samples[:,0]**2))
print('variance in y-direction:', np.mean(samples[:,1]**2))
print('covariance of x and y:', np.mean(samples[:,0]*samples[:,1]))

Compare to the covariance matrix we defined above. 

Now let's see what happens if we go to higher dimensions. For simplicity, we use a symmetric Gaussian without correlations.

In [ ]:
#Define the D-dimensional Gaussian:
sigma = 2. #standard deviation in all dimensions
def DdimGauss(xvec,sigma,D):
    det = sigma**(2*D)
    norm = 1./(2.*np.pi)**(D/2.)/det**0.5
    return norm*np.exp(-0.5*np.dot(xvec,xvec/sigma**2))

#Define the drawing step:
def draw_sample_Ddim(D, rng = np.random.default_rng()):
    return rng.uniform(low = -10, high = 10, size = D) #D-dimensional uniform distribution

In [ ]:
#Draw some samples:
N = 10000 #number of samples to draw
count = 0 #count the accepted samples
samples = [] #store the accepted samples
D = 3

#Here we are cheating a bit: Since we know what the maximum of the D-dimensional PDF is, we can
#calculate the optimal value of c to use:
c = DdimGauss(np.zeros(D),sigma,D)*20.**D #the Gaussian reaches its max value at the mean (= 0). 

for i in range(N):
    samp = draw_sample_Ddim(D)
    acc = AcceptReject(c,D,DdimGauss,samp,sigma,D)
    if acc:
        count += 1
        samples.append(samp)

print('acceptance ratio:', count/N)

Rerun the cell for different choices of the dimension D. What happens to the acceptance ratio?

This problem is known as "the curse of dimensionality". What do you think we could do to increase the acceptance ratio?

Until now we have been drawing samples randomly in some range without using any 'past experience' of where samples have been accepted.

## Markov Chain Monte Carlo: Metropolis Hastings
[See Metropolis et al. (1953)](https://bayes.wustl.edu/Manual/EquationOfState.pdf)

Markov Chain: Draw random samples in an ordered chain:

* "Markov" property: Each sample depends on the sample before, but not on any other samples
* Pro: Curse of dimensionality less severe
* Con: Successive samples are correlated -> need to discard many of them to obtain a set of independent samples

Specific example: Metropolis-Hastings for drawing from distribution $P$.

* Select a "proposal distribution" $Q(x'|x)$ to draw a new sample $x'$ given a previous sample $x$. Below, we use a Gaussian distribution centered at $x$ for the proposal distribution. 
* If $P(x') \geq P(x)$, accept the new sample (we are going "uphill"). 
* If $P(x') < P(x)$, accept the new sample with probability $(P(x')/P(x))(Q(x|x')/Q(x'|x))$. Note that if Q is Gaussian, this simplifies to $P(x')/P(x)$ because a Gaussian is symmetric.  
* If a sample is rejected, the chain stays at the same position (i.e., the old sample is repeated).
* Note that the normalization of $P$ doesn't matter. Very helpful when we don't know the normalization! 

Let's try again to sample from a two-dimensional Gaussian, this time with MCMC.

In [ ]:
plt.imshow(G,cmap=plt.cm.Blues,extent=[xvals.min(),xvals.max(),yvals.max(),yvals.min()])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(label = r'$p(x,y)$')
plt.title('target distribution')
plt.show()

In [ ]:
#Define the proposal step (we are using a symmetric Gaussian proposal density in D dimensions):
def proposal(oldsamp, sigmaprop, D, rng = np.random.default_rng()):
    '''Propose a new sample based on the current sample according to a symmetric Gaussian proposal pdf
    
    inputs:
    -oldsamp - current sample, D-dimensional array
    -sigmaprop - standard deviation (in all dimensions) of the Gaussian proposal 
    -D - number of dimensions
    -rng - random number generator

    returns:
    -newsamp - proposed new sample, D-dimensional array
    '''
    newsamp = oldsamp + sigmaprop * rng.normal(size = D)
    return newsamp

#Define the acceptance-rejection step, return the new sample and a boolean that tells us whether
#or not the new sample was accepted):
def accept(rng, newsamp,oldsamp,paramrange,P,*Pargs):
    '''acceptance-rejection step of MCMC
    
    inputs:
    -rng -- random number generator
    -newsamp -- proposed new sample, D-dimensional array
    -oldsamp -- old sample, D-dimensional array
    -paramrange -- acceptable range of samples; array with shape (D, 2)
    -P -- target probability distribution to draw from (doesn't have to be normalized); function
    -*Pargs -- variable length argument, parameters that define target P

    returns:
    -acc -- a boolean that tells us whether new sample was accepted
    -next sample in the chain (either oldsamp or newsamp) -- D-dimensional array
    '''
    #first check if newsamp is in desired range, i.e.
    #if every dim of newsamp > lower bound in paramrange
    #and if every dim of new samp < upper bound in paramrange
    if not ((np.array([p1 - p2 for p1, p2 in zip(newsamp, np.transpose(paramrange)[:][0])])>0).all() \
            and (np.array([p1 - p2 for p1, p2 in zip(np.transpose(paramrange)[:][1],newsamp)])>0).all()): 
        acc = False #if newsamp is not in desired range paramrange, reject (acc = False) and return old sample
        return acc, oldsamp

    #compute target distribution at proposed sample (newsamp) and and current sample (oldsamp)
    newprob = P(newsamp,*Pargs) 
    oldprob = P(oldsamp,*Pargs) 
    
    #if proposed sample is "uphill" -- accept 
    if newprob >= oldprob:
        acc = True 
        return acc, newsamp

    #if not, accept with probability newprob/oldprob
    else:
        prob = newprob/oldprob
        acc = rng.choice([True,False],p=[prob,1-prob])
        return acc, acc*newsamp + (1 - acc)*oldsamp #Note that this is either newsamp or oldsamp

#Define function that runs an entire chain:
def run_chain(rng, steps,paramrange,sigmaprop,D,P,*Pargs):
    '''Runs an MCMC chain of samples from target distribution

    inputs:
    -rng -- random number generator
    -steps -- number of steps to take in chain; int
    -paramrange -- acceptable range of samples; array with shape (D, 2). In dimension i, paramrange[i][0] gives lower bound and paramrange[i][1] gives upper bound.
    -sigmaprop -- standard deviation of symmetric Gaussian proposal distribution; float
    -D -- number of dimensions; int
    -P -- target probability distribution to draw from (doesn't have to be normalized); function
    -*Pargs -- variable length argument, parameters that define target P

    output:
    -samples -- samples from target distribution; array of length steps
    -ar --  acceptance ratio; float
    '''
    oldsamp=np.array([rng.uniform(paramrange[d][0],paramrange[d][1]) for d in range(D)])#Draw a random starting point in acceptable range
    count = 0 #Count the number of accepted samples
    samples = [oldsamp] #Store all samples
    for i in range(steps):
        newsamp = proposal(oldsamp,sigmaprop,D, rng) #Propose a new sample
        acc, newsamp = accept(rng, newsamp,oldsamp,paramrange,P,*Pargs) #decide whether or not to accept it
        samples.append(newsamp) #Add the sample to the list of samples
        if acc:
            count += 1
        oldsamp = newsamp #Move to the new sample
    ar = count/steps #compute the acceptance ratio
    return np.array(samples), ar


Now let's run our chain. Recall we are trying to draw samples from a twoDGauss with covariance matrix defined above as cov.

In [ ]:
rng = np.random.default_rng(seed = 13434)

Nsteps = 5000 #number of steps to run the chain for
paramrange = np.transpose(np.array([[-10]*D,[10]*D])) #acceptable range to sample in
sigmaprop = 1 #width of the proposal distribution
D = 2 #dimension of the parameter space
Ptarget = twoDGauss #target distribution
#Pargs here is cov, invcov
samples, ar = run_chain(rng, Nsteps,paramrange,sigmaprop,D,Ptarget,cov,invcov) #run the chain
print('acceptance ratio:', ar)

In [ ]:
#Plot the chain on top of the 2D-density:
plt.imshow(G,cmap=plt.cm.Blues,extent=[xvals.min(),xvals.max(),yvals.max(),yvals.min()])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(label = 'target density')
plt.plot(samples[:,0],samples[:,1],marker = '.', ls = '-',color='red', alpha = 0.1)
plt.plot(samples[0,0], samples[0,1], marker = '*', color = 'black') #first sample
plt.show()

Plot the x values and y values of the samples separately as a function of step number.

In [ ]:
plt.plot(samples[:,0])
plt.xlabel(r'sample number')
plt.ylabel(r'$x$')
plt.show()

plt.plot(samples[:,1])
plt.xlabel(r'sample number')
plt.ylabel(r'$y$')
plt.show()

This is called a "Markov chain."

Note that:
* it takes the chain some time to "burn in" and forget its initial value. We can throw away the first several samples.
* Even after burn-in, the samples are not independent. We can keep only 1 in every N samples where N is the correlation length.

In [ ]:
#use the plot to determine how many samples to throw out as "burn-in":
burnin = 100
non_burnin_samples = samples[burnin:]

#calculate the auto-correlation of the remaining samples (in the x dimension):
xmean=(np.mean(non_burnin_samples[:,0]))
xvar=(np.var(non_burnin_samples[:,0]))

Nsamps_non_burnin = len(non_burnin_samples[:,0])
#for every h, calculate the normalized correlation between samples "h" apart.
#This is Cov(X_i, X_{i+h})/xvar, or approximately
#sum_{i=1}^{N-h}((samp[i+h]-mean)*(samp[i]-mean)) / (N-h)/ xvar
#we don't want to consider h that is too high because then we have a very noisy estimate. 
autocorr = np.array([np.sum((non_burnin_samples[h:,0]-xmean)*(non_burnin_samples[:-h,0]-xmean))/(Nsamps_non_burnin-h)/xvar for h in range(1,100)])                              

plt.plot(autocorr)
plt.xlabel(r'difference in sample number')
plt.ylabel(r'correlation')
plt.show()

We can estimate the integrated autocorrelation time as 1 + 2*(the sum of autocorr over h). See, for example, the explanation in the [emcee tutorial](https://emcee.readthedocs.io/en/stable/tutorials/autocorr/). 

In [ ]:
autocorr_time = 1+2*np.sum(autocorr)
autocorr_time = round(autocorr_time)
print(f'autocorrelation time is {autocorr_time}')

We want to keep only the "independent samples," throwing away those ones that are within an autocorrelation time of each other.

In [ ]:
independentsamples = non_burnin_samples[::autocorr_time]
print(f"we only have {independentsamples.shape[0]} independent samples")
print('effective acceptance ratio:', independentsamples.shape[0]/Nsteps)

#Plot only the independent samples:
plt.imshow(G,cmap=plt.cm.Blues,extent=[xvals.min(),xvals.max(),yvals.max(),yvals.min()])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(label = "target density")
plt.plot(independentsamples[:,0],independentsamples[:,1],'.',color='red')
plt.show()

We have increased the acceptance ratio compared to the rejection sampling, but we have decreased the relative number of independent samples.

A high acceptance ratio isn't always desirable. Can you think of any other reason why a high acceptance ratio might not be ideal?

### Let's try a multimodal distribution. 

In [ ]:
#Define a multimodal distribution (remember that the normalization doesn't matter):
def multimodal(samp,cov,invcov,mu2 = 5):
    return twoDGauss(samp,cov,invcov) + 0.2*twoDGauss(np.array(samp)-mu2, cov, invcov)

#Plot it:
mm = multimodal([gridx,gridy],cov,invcov)
plt.imshow(mm,cmap=plt.cm.Blues,extent=[xvals.min(),xvals.max(),yvals.max(),yvals.min()])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar()
plt.show()

In [ ]:
rng = np.random.default_rng(seed = 13434) # first try this
#rng = np.random.default_rng(seed = 9230) # then try this

Nsteps = 10000 #number of steps to run the chain for

Ptarget = multimodal #target distribution
samples, ar = run_chain(rng, Nsteps,paramrange,sigmaprop,D,Ptarget,cov,invcov) #run the chain
print('acceptance ratio:', ar)

In [ ]:
#Plot the chain on top of the 2D-density:
plt.imshow(mm,cmap=plt.cm.Blues,extent=[xvals.min(),xvals.max(),yvals.max(),yvals.min()])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.colorbar(label = 'target density')
plt.plot(samples[:,0],samples[:,1],marker = '.', ls = '-',color='red', alpha = 0.05)
plt.plot(samples[0,0], samples[0,1], marker = '*', color = 'black') #first sample
plt.show()

Use the different seeds provided above. What do you notice? Does this change if you draw more random samples? Remember: an MCMC is guaranteeed to converge to the true distribution, but it is not guaranteed to do so fast.

Marginalizing over parameters is trivial; just ignore them.

In [ ]:
independent_samples = samples[100::40] #remove burn-in phase and correlated samples

x_values = independent_samples[:,0]

plt.hist(x_values,range=(-10,10),bins=20,density=True)
plt.xlabel(r'$x$')
plt.ylabel(r'$p(x)$')
plt.show()

y_values = independent_samples[:,1]

plt.hist(y_values,range=(-10,10),bins=20,density=True)
plt.xlabel(r'$y$')
plt.ylabel(r'$p(y)$')
plt.show()

A good way to visualize both the joint and marginal distributions is with a corner plot:

In [ ]:
import corner

corner.corner(independent_samples, labels = ['x', 'y']);

Now let's try again going to higher dimensions.

In [ ]:
#Standard deviation in each direction for the D-dimensional Gaussian:
sigma = 2.

#Run a chain:
rng = np.random.default_rng(seed = 43895)
D = 10 #10 dimensions
Nsteps = 5000
sigmaprop = 1
samples, ar = run_chain(rng,Nsteps,np.transpose(np.array([[-10]*D,[10]*D])),sigmaprop,D,DdimGauss,sigma,D)
print('acceptance ratio:', ar)

In [ ]:
#Plot the first dimension of the samples:
plt.plot(samples[:,0])
plt.xlabel(r'sample number')
plt.ylabel(r'$x$')
plt.show()

#make a corner plot (the most boring corner plot ever)
corner.corner(samples);